In [1]:
!pip install qiskit

In [2]:
!pip install qiskit_aer

In [3]:
!pip install pylatexenc

# Imports

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import *
from pylatexenc import *

# Quantum one Time Pad

In [5]:
def qunatum_one_time_pad(message):

    backend = Aer.get_backend('qasm_simulator') 

    # Generate random key
    key = ''
    for _ in range(len(message)):
        # QRNG to generate random key
        qc = QuantumCircuit(2, 2)
        qc.h(0)  
        qc.h(1)
        qc.measure([0, 1], [0, 1])    
        result = backend.run(qc, shots = 1).result()
        counts = result.get_counts()
        fig = qc.draw(output='mpl')
        fig.savefig("QRNG.png")
        random_pair = list(counts.keys())[0]
        # Reverse due to qiskit bit order (LSB comes first)
        key += random_pair[::-1]  

    print("Random key:", key)

    # Encryption
    qc_enc = QuantumCircuit(len(message), len(message))
    for bit in range(len(message)):
        # If the bit is '1', flip the qubit from |0> to |1>
        if message[bit] == '1':
            qc_enc.x(bit)  
        #If first key bit is '1', Z gate is applied (pk[0], pk[2], .....)
        if key[2*bit] == '1':
            qc_enc.z(bit) 
        # If second key bit is '1', X gate is applied (pk[1], pk[3], .....)
        if key[2*bit + 1] == '1':
            qc_enc.x(bit)  
            
    # Measure encryption
    qc_enc.measure(range(len(message)), range(len(message)))
    result_enc = backend.run(qc_enc, shots=1024).result()
    counts_enc = result_enc.get_counts()
    encrypted_bits = list(counts_enc.keys())[0][::-1]
    print("Encrypted bits:", encrypted_bits)
    fig = qc_enc.draw(output='mpl')
    fig.savefig("Encryption_Circuit.png")
    
    # Decryption
    qc_dec = QuantumCircuit(len(message), len(message))
    
    # Initialize qubits to encrypted bits
    for i, bit in enumerate(encrypted_bits):
        if bit == '1':
           qc_dec.x(i)
            
    for bit in range(len(message)):
        # Apply Z gate to decrypt (all gates are reversible)
        if key[2*bit] == '1':
            qc_dec.z(bit)   
        # Apply X gate to decrypt (all gates are reversible)
        if key[2*bit + 1] == '1':
            qc_dec.x(bit)  

    # Measure decryption
    qc_dec.measure(range(len(message)), range(len(message)))
    result_dec = backend.run(qc_dec, shots=1024).result()
    counts_dec = result_dec.get_counts()
    decrypted_bits = list(counts_dec.keys())[0][::-1]
    print("Decrypted bits:", decrypted_bits)
    fig = qc_dec.draw(output='mpl')
    fig.savefig("Decryption_Circuit.png")

In [10]:
qunatum_one_time_pad('1001')

Random key: 11011000
Encrypted bits: 0101
Decrypted bits: 1001
